In [1]:
import pandas as pd
from itertools import product
from tqdm.notebook import tqdm

import sys, os

sys.path.append(os.path.abspath('..'))
%load_ext autoreload
%autoreload 2
from modules.config import *
from modules.storage import (
    get_demand_model_data,
    store_results
)
from modules.svm import *

In [2]:
def get_svm_metas():
    return [
        {'kernel': ['linear'], 'C': [1, 10, 100],       'gamma': [-1],                          'degree': [-1],         'max_iter': [1000000]},
        {'kernel': ['rbf'],    'C': [0.1, 1, 10, 100],  'gamma': [0.1, 0.01, 0.001, 0.0001],    'degree': [-1],         'max_iter': [1000000]},
        {'kernel': ['poly'],   'C': [1, 10, 100, 1000], 'gamma': [-1],                          'degree': [2, 3, 4, 5], 'max_iter': [1000000]}
    ]

In [3]:
def execute_stage(path, h3_res, time_interval_length, get_available_model_metas_for_stage, do_evaluate_model, silent):
    all_possible_metas = get_svm_metas()
    metas = get_available_model_metas_for_stage(h3_res, time_interval_length, all_possible_metas)
    
    iterator = metas if silent else tqdm(metas)
    for param_grid in iterator:
        if not silent:
            feedback = f"h3: {h3_res} | t:{time_interval_length} | - " + param_grid[0]["kernel"][0]
            tqdm.write(feedback, end="\r")
        
        model_data = get_demand_model_data(h3_res, time_interval_length)
        # model_data = model_data.sample(200000)

        X_train, X_test, y_train, y_test = split_and_scale_data(model_data, "outcome")
        models = train_model(param_grid, X_train, y_train)
        results = get_results(models, h3_res, time_interval_length, do_evaluate_model, X_test, y_test)
        store_results(results, path)  
        
        if not silent:
            tqdm.write(feedback + " ✓")

In [4]:
execute_stage(
    SVM_FIRST_STAGE_RESULTS_PATH,
    TUNE_H3_RESOLUTION,
    TUNE_TIME_INTERVAL_LENGTH,
    get_availabe_models_metas_first_stage,
    do_evaluate_model=False,
    silent=False,
)

  0%|          | 0/1 [00:00<?, ?it/s]

h3: 8 | t:6 | - rbf ✓


In [17]:
results.sort_values(by='mean_test_score', ascending=False).groupby('param_kernel').mean_test_score.first()

param_kernel
linear   -1.660247
poly     -1.508812
rbf      -1.465577
Name: mean_test_score, dtype: float64

In [6]:
results.groupby('param_kernel').mean_fit_time.sum() / 60

param_kernel
linear     57.638013
poly      476.428271
rbf       201.699398
Name: mean_fit_time, dtype: float64

In [6]:
results = pd.read_parquet(SVM_FIRST_STAGE_RESULTS_PATH)
results.sort_values(by=['mean_test_score'], ascending=False)

,iter,n_resources,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,param_max_iter,params,...,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score,n_iter,h3_res,time_interval_length,param_gamma,param_degree
4,0,8290,54.808515,0.629360,11.699974,0.465198,1.0,rbf,1000000,"{'C': 1.0, 'degree': None, 'gamma': 0.01, 'ker...",...,-1.240730,-1.419944,-1.493995,-1.319479,0.116704,434001,8,6,0.0100,NaN
7,0,8290,61.887783,2.143711,13.117010,0.575450,10.0,rbf,1000000,"{'C': 10.0, 'degree': None, 'gamma': 0.01, 'ke...",...,-0.968602,-1.101078,-1.162574,-1.023994,0.091734,0,8,6,0.0100,NaN
11,0,8290,61.977852,1.651096,8.008226,0.643548,100.0,rbf,1000000,"{'C': 100.0, 'degree': None, 'gamma': 0.001, '...",...,-1.251774,-1.431241,-1.494112,-1.327835,0.114344,0,8,6,0.0010,NaN
21,0,8290,72.496359,2.332309,11.504219,0.713604,10.0,poly,1000000,"{'C': 10.0, 'degree': 2.0, 'gamma': None, 'ker...",...,-1.278043,-1.453530,-1.513180,-1.352580,0.110961,0,8,6,NaN,2.0
16,2,74610,9881.552610,513.575073,284.903122,5.123617,10.0,rbf,1000000,"{'C': 10.0, 'degree': None, 'gamma': 0.01, 'ke...",...,-1.367001,-1.403957,-1.402731,-1.397316,0.020037,0,8,6,0.0100,NaN
50,1,74613,12912.278346,861.005329,1010.193051,78.991467,1.0,rbf,1000000,"{'C': 1.0, 'degree': None, 'gamma': 0.1, 'kern...",...,-1.224784,-1.263204,-1.260707,-1.255135,0.019801,0,8,6,0.1000,NaN
45,0,24871,567.399394,7.833903,129.166120,7.621170,1.0,rbf,1000000,"{'C': 1.0, 'degree': None, 'gamma': 0.1, 'kern...",...,-1.048093,-1.111682,-1.107257,-1.085374,0.039947,0,8,6,0.1000,NaN
22,0,8290,55.815001,2.124346,8.583216,1.233744,10.0,poly,1000000,"{'C': 10.0, 'degree': 3.0, 'gamma': None, 'ker...",...,-1.285417,-1.395458,-1.455411,-1.332355,0.080761,0,8,6,NaN,3.0
27,0,8290,46.836145,1.878005,6.990796,1.143006,100.0,poly,1000000,"{'C': 100.0, 'degree': 4.0, 'gamma': None, 'ke...",...,-1.129696,-1.124297,-1.178510,-1.116729,0.040064,0,8,6,NaN,4.0
25,0,8290,230.725443,8.757444,8.920481,1.452884,100.0,poly,1000000,"{'C': 100.0, 'degree': 2.0, 'gamma': None, 'ke...",...,-1.235170,-1.398997,-1.474063,-1.304200,0.112618,0,8,6,NaN,2.0


In [6]:
len(get_demand_model_data(TUNE_H3_RESOLUTION, TUNE_TIME_INTERVAL_LENGTH))

373066

In [7]:
# resolutions = list(product(PREDICTIVE_H3_RESOLUTIONS, CALC_TIME_INTERVAL_LENGTHS))
# resolutions = resolutions + (ADDITIONAL_PREDICTIVE_RESOLUTIONS)
# resolutions

In [8]:
# for h3_res, time_interval_length in tqdm(resolutions):
#     execute_stage(
#         SVM_SECOND_STAGE_RESULTS_PATH,
#         h3_res,
#         time_interval_length,
#         get_availabe_models_metas_second_stage,
#         do_evaluate_model=True,
#         silent=True,
#     )

In [9]:
# results = pd.read_parquet(SVM_FIRST_STAGE_RESULTS_PATH)
# results.sort_values(by=['mean_train_score'], ascending=False).head(2)

In [10]:
# results_final = pd.read_parquet(SVM_SECOND_STAGE_RESULTS_PATH)
# results_final